In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

train_transformed = pd.read_csv("../input/saso-avito-demand-prediction-01fe96/train_transformed.csv")
test_transformed = pd.read_csv("../input/saso-avito-demand-prediction-01fe96/test_transformed.csv")
test_transformed.head(5)
# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle


X,y = shuffle(train_transformed.iloc[:, [3,4,5,6,7,8,9,12,14,17]], train_transformed.iloc[:,[18]], random_state=13)
#X, y = shuffle(boston.data, boston.target, random_state=13)
X = X.fillna(value=0)
X = X.astype(np.float32)
y = y.values.ravel()

offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

# #############################################################################
# Fit regression model
params = {'n_estimators': 500, 'max_depth': 5, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'lad'}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)
mse = mean_squared_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)

# #############################################################################
# Plot training deviance

# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(clf.staged_predict(X_test)):
    test_score[i] = clf.loss_(y_test, y_pred)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, clf.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')



In [ ]:
deal_probability = clf.predict(test_transformed.iloc[:, [3,4,5,6,7,8,9,12,14,17]].fillna(value=0))
#submission_op = pd.DataFrame(user_id = test_transformed['user_id'], deal_probability=deal_probability)
submission_op = pd.DataFrame({'item_id': test_transformed['item_id'], 'deal_probability': deal_probability})
submission_op = submission_op[['item_id', 'deal_probability']]
submission_op.to_csv('submission.csv', index=False)
